# TP11 (Student version): Pagerank

We can use the following libraries.

In [1]:
import matplotlib.pyplot as plt
import math
import sys
import random
import time
import copy
print(sys.version)

3.7.3 (default, Jul 25 2020, 13:03:44) 
[GCC 8.3.0]


## Exercise 1: preliminary questions

In this TP, we will use the graph http://lioneltabourier.fr/documents/wiki_2009.txt. It is a subpart of the English language Wikipedia collected in 2009. A link represent a hyperlink from a page to another.

**Warning:** it is a directed graph, so in this case a line

12 126

means that there is a directed link from node 12 to node 126, but not necessarily in the other direction!

For your information, we indicate that this network has

- 50988 nodes

- 1778125 directed links

### Question 1

Load the graph in memory, in the adjacency list format, **for both the list of predecessors and the list of successors**. 

Check its number of nodes and of directed links. 

In [17]:
def load_graph(graph_file):
    predecessors = {}
    successors = {}
    with open(graph_file, "r") as file:
        for line in file:
            node1, node2 = [int(a) for a in line.split()]
            if node1 not in successors:
                successors[node1] = []
            successors[node1].append(node2)
            if node2 not in predecessors:
                predecessors[node2] = []
            predecessors[node2].append(node1)
    return predecessors, successors

def count_node(graph):
    return len(graph)
def count_link(graph):
    count = 0
    for node in graph:
        count += len(graph[node])
    return count

In [22]:
graph_file_path = "graphs/wiki_2009.txt"
predecessors, successors = load_graph(graph_file_path)

print("Nombre de noeuds dans predecessors : {}".format(count_node(predecessors)))
print("Nombre de noeuds dans successors : {}".format(count_node(successors)))
print("Nombre de liens dans predecessors : {}".format(count_link(predecessors)))
print("Nombre de liens dans successors : {}".format(count_link(successors)))
print("Nombre de noeuds total : {}".format(len(set(predecessors.keys()).union(set(successors.keys())))))


Nombre de noeuds dans predecessors : 47424
Nombre de noeuds dans successors : 50702
Nombre de liens dans predecessors : 1778125
Nombre de liens dans successors : 1778125
Nombre de noeuds total : 50988



We remind you that the transition matrix $T$ is defined like this: if there is a link from $u$ to $v$ then $T[u][v] = \frac{1}{d_{out}(u)}$ where $d_{out}(u)$ is the out-degree of $u$ and otherwise  $T[u][v] = 0$.

Note that it is not possible to store $T$ in memory as a $ n \times n $ matrix, it would take too much memory. So instead of explicitly computing a matrix $T$, we use the adjacency lists of the graph (lists of predecessors, lists of successors) in this way: 

- from the list of successors, we store the outdegree ($ d_{out}$) of the nodes in a dedicated dictionary,

- we use that a node $v$ receive PageRank from node $u$ if and only if $u$ is a predecessor of $v$.


### Question 2

Create a dictionary that contains the out-degree of each node. Note that to avoid problems later, you should give the out-degree of all the nodes if the network, even if it is $0$. 

In [24]:
def out_degrees(predecessors, successors):
    my_out_degree = {}
    for node in predecessors: # out-degree of node that may not exist in successors
        my_out_degree[node] = 0
    for node in successors: # out-degree of node that have successors
        my_out_degree[node] = len(successors[node])
    return my_out_degree 
        

In [37]:
from json import dumps
my_out_degrees = out_degrees(predecessors, successors)
print(dumps(my_out_degrees, indent = 4))

{
    "627": 136,
    "728": 25,
    "1023": 113,
    "1793": 281,
    "1990": 274,
    "2287": 19,
    "3271": 41,
    "4443": 140,
    "4705": 25,
    "4927": 38,
    "5708": 39,
    "5734": 12,
    "6258": 293,
    "6512": 64,
    "6675": 87,
    "7935": 16,
    "8144": 256,
    "9235": 17,
    "9567": 15,
    "9764": 66,
    "10030": 58,
    "10597": 135,
    "10930": 246,
    "11054": 129,
    "11109": 31,
    "11188": 128,
    "11282": 32,
    "11826": 98,
    "11887": 97,
    "11891": 101,
    "12229": 75,
    "12557": 58,
    "12710": 40,
    "12719": 32,
    "13314": 34,
    "13784": 38,
    "14936": 46,
    "15181": 43,
    "15936": 291,
    "15941": 71,
    "15983": 273,
    "16205": 24,
    "16743": 140,
    "17626": 46,
    "17629": 37,
    "17864": 33,
    "17865": 44,
    "18048": 113,
    "18499": 121,
    "20168": 26,
    "20217": 48,
    "21634": 48,
    "21763": 305,
    "22568": 266,
    "22677": 15,
    "23040": 132,
    "23626": 98,
    "23627": 86,
    "24902": 3

### Question 3

Using the previous questions, implement a basic power iteration (for the moment, it is not a complete PageRank algorithm). 

The principle is to iterate $t$ times the matrix product:

$$ X \leftarrow T.X $$

$X$ is a vector initialized to $ [\frac{1}{n} \ldots \frac{1}{n}]$ and $T$ is the transition matrix. We strongly advise you to store $X$ as a dictionary of float.

Run the power iteration for $ t=10 $ steps and measure at each step the value of $ ||X||_1 = \sum _i |X[i]| $.
What do you observe? Can you explain in one sentence?

In [38]:
def init_vector(predecessors, successsors):
    x = {}
    for node in predecessors:
        x[node] = 0
    for node in successsors:
        x[node] = 0
    count = len(x)
    for node in x:
        x[node] = 1 / count
    return x
    
def power_iteration(my_out_degrees, predecessors, x):
    new_x = {}
    for node in x:
        new_x[node] = 0
        if node in predecessors: # those without predecessors wont have anithing at the end
            for node_pred in predecessors[node]:
                new_x[node] += x[node_pred] / my_out_degrees[node_pred]
        else:
            new_x[node] = 0
    return new_x

In [41]:
predecessors, successors = load_graph("graphs/wiki_2009.txt")
x = init_vector(predecessors, successors)
my_out_degrees = out_degrees(predecessors, successors)
for t in range(10):
    print(sum(x.values()))
    x = power_iteration(my_out_degrees, predecessors, x)

1.0000000000001132
0.994390837059708
0.9936039832167154
0.9929045735798405
0.9922735463834518
0.9916499712662787
0.9910335666804821
0.9904190411990255
0.9898061668615519
0.9891941066835478


## Exercise 2: Pagerank with evaporation

### Question 4

Now, we improve the basic power iteration process to make a real Pagerank program, by adding a renormalization and an evaporation (or teleportation) process.

So now each iteration is described by:

$$ X \leftarrow (1-s).T.X + s.I $$

where $I$ is the vector $ [\frac{1}{n} \ldots \frac{1}{n}]$.

Implement the Pagerank alogorithm (this is Algorithm 3 in the course). Don't forget to renormalize $X$ after each step, that is to say do: $ X[i] \leftarrow \frac{X[i]}{||X||_1}$.

Run the Pagerank for $t=10$ steps, $s=0.2$.

Observe the nodes which have the top-5 pagerank values, then go and look in this document http://lioneltabourier.fr/documents/index_wiki_2009.txt to see to what Wikipedia pages they correspond. 

Comment your result: do you think it is surprising?

In [46]:
def pagerank(my_out_degrees, predecessors, successors, x, s = 0.2):
    v_init = init_vector(predecessors, successors)
    new_x = {}
    for node in x:
        new_x[node] = 0
        if node in predecessors: # those without predecessors wont have anithing at the end
            for node_pred in predecessors[node]:
                new_x[node] += x[node_pred] / my_out_degrees[node_pred]
        else:
            new_x[node] = 0
        new_x[node] = new_x[node] * (1 - s) + s * v_init[node]
    # normalization
    norm = sum(new_x.values())
    for node in new_x:
        new_x[node] /= norm
    return new_x
    

In [51]:
predecessors, successors = load_graph("graphs/wiki_2009.txt")
x = init_vector(predecessors, successors)
my_out_degrees = out_degrees(predecessors, successors)
s = 0.2
steps = 10
for t in range(steps):
    #print(sum(x.values()))
    x = pagerank(my_out_degrees, predecessors,successors, x, s)
sorted_list = sorted(x.items(), key=lambda x: x[1], reverse=True)
for i in range(5):
    print(sorted_list[i])

(31717, 0.0022533650878698184)
(11867, 0.0017397681676120408)
(32927, 0.0017021277717283799)
(36164, 0.0016649663921986108)
(9316, 0.0016156420823766346)


A travers l'algorithme, nous affectons une valeur à chaque noeud (page Wikipedia), plus cette valeur est élevé, plus cela veut dire que cette page est importante/référencé. 